## TODO:
1. Add RGB channels for $S_0$

In [13]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
from dtcwt_classifier import DtcwtClassifier
from keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

cls = DtcwtClassifier(m = 1)
cls.fit(X_train[0:100], y_train[0:100])

Train on 100 samples
100/100 [==============================] - 1s 11ms/sample - loss: 11.1194 - accuracy: 0.1200


In [16]:
_, accuracy = cls.evaluate(X_test[0:100], y_test[0:100])
print('Accuracy: %.2f' % (accuracy*100))

100/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================